In [2]:
from ngram_tools.download_ngrams import download_ngram_files
from ngram_tools.convert_to_jsonl import convert_to_jsonl_files
from ngram_tools.lowercase_ngrams import lowercase_ngrams
from ngram_tools.lemmatize_ngrams import lemmatize_ngrams
from ngram_tools.filter_ngrams import filter_ngrams
from ngram_tools.sort_ngrams import sort_ngrams
from ngram_tools.consolidate_ngrams import consolidate_duplicate_ngrams
from ngram_tools.make_yearly_files import make_yearly_files

from ngram_tools.helpers.verify_sort import check_file_sorted
from ngram_tools.helpers.print_jsonl_lines import print_jsonl_lines

# **Process Multigrams for Training Word-Embedding Models**

## **Goal**: Download and preprocess mulitgrams for use in training `word2vec` models. 

This workflow is resource-intensive and is probably only practical when run on a computing cluster. On my university's High Performance Computing (HPC) cluster, I request the maximum 14 cores (48 logical processors) and 128G of memory and use a 2T fast-I/O NVMe SSD filespace—and I still run up against time and resource limits. I've designed the code to be efficient, although further optimization is surely possible.

The code affords options to keep conserve resources. Throughout the workflow you can specify `compress=True`, which tells a script to compress its output files. In my experience, there is little downside to using LZ4 compression, since it's very fast and cuts file sizes by about half. Downstream modules will see the `.lz4` extensions and handle the files accordingly. If you know your workflow runs correctly and wish to further conserve space, you can specify `delete_input=True` for many of the scripts; this will delete the source files for a given step once it is complete. The scripts are fairly memory-efficient—with the exception of `sort_ngrams` and `index_and_create_vocab_files`, which sort multiple files in memory at once. When processing multigrams, I've found that allocating more than ~10 workers in these scripts leads to memory exhaustion (with 128G!) and slow processing.

**NOTE:** You'll probably want to have run `workflow_unigrams.ipynb` before processing multigrams. That workflos allows you create a vocabulary file for filtering out uncommon tokens from the multigrams. Although you can run the `filter_ngrams` module without a vocab file, most use cases will call for one.

### Download multigrams
Here, I'm using `download_ngrams` module to fetch 5grams appended with part-of-speech (POS) tags (e.g., `_VERB`). Although you can specify `ngram_type='untagged'`, POS tags are necessary to lemmatize the tokens. Specify the number of parallel processes you wish to use by setting `workers` (the default is all available processors). You may wish to specify `compress=True` becausae 5gram files are _big_.

`[Runtime with compression:  4:19:11.661609]`

In [ ]:
download_ngram_files(
    ngram_size=3,
    ngram_type='tagged',
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    compress=False,
    overwrite=False
)

### Convert files from TXT to JSONL
This module converts the original multigram files' text data to a more flexible JSON Lines (JSONL) format. Although this increases storage demands, it makes downstream processing more efficient.

`[Runtime with compression:  1:28:52.759835]`

In [6]:
convert_to_jsonl_files(
    ngram_size=3,
    ngram_type='tagged',
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    compress=False,
    overwrite=False,
    delete_input=True
)

Start Time:                2025-01-24 12:43:24.013027

Conversion Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng/3gram_files/1download
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/3gram_files/2convert
File index range:          0 to 457
Files available:           458
Files to use:              458
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng/3gram_files/1download/3-00000-of-06881.txt
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/3gram_files/1download/3-00529-of-06881.txt
Ngram size:                3
Ngram type:                tagged
Number of workers:         48
Compress output files:     False
Overwrite existing files:  False
Delete input directory:    True

Converting      |██████████████████████████████████████████████████| 100.0% 458         /458        

End Time:                  2025-01-24 12:46:37.505712
Total runtime:             0:03:13.492685


### Make multigrams all lowercase
This module lowercases all characters in the multigrams. Most use cases benefit from this.

`[Runtime with compression:  0:47:39.944151]`

In [ ]:
lowercase_ngrams(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    compress=True,
    overwrite=False,
    delete_input=True
)

### Lemmatize the multigrams
Likewise, most use cases will benefit from multigrams that are lemmatized—that is, reduced to their base form. This requires POS-tagged multigrams. Example: `people_NOUN` ("the people of this land") will be converted to `person` in the output; `people_VERB` ("to people this land") will not. The POS tag will then be discarded as it is no longer useful.

`[Runtime with compression:  1:39:30.015704]`

In [ ]:
lemmatize_ngrams(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    compress=True,
    overwrite=False,
    delete_input=True
)

### Filter the multigrams
This module removes tokens that provide little information about words' semantic context—specifically, those that contain numerals (`numerals=True`), nonalphabetic characters (`nonalpha=True`), stopwords (high-frequency, low information tokens like "the" and "into"; `stops=True`), or short words (those below a certain user-specified character count; here, `min_token_length=3`). You can also specify a **vocabulary file** like the one illustrated in the unigram workflow. A vocabulary file is simply a list of the _N_ most common words in the unigram corpus; the multigram tokens are checked against this list and those that don't appear in it are dropped.

The filtering process will inevitably turn some longer ngrams (e.g., 5grams) into shorter ones (e.g., 3grams) after unwanted tokens are dropped. The training of word-embedding models requires _linguistic context_—which in turn requires ngrams containing more than one token. A unigram isn't useful for helping a model learn what "company" a word keeps. Thus, the `min_tokens` option allows you to drop ngrams that fall below a specified length during filtering. If filtering results in an ngram with fewer than the minimum tokens, all data for that ngram is dropped entirely. Here, I've set `min_tokens=2`, since two tokens (and higher) provide at least some contextual information.

`[Runtime with compression:  0:49:24.366522]`

In [ ]:
filter_ngrams(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    numerals=True,
    nonalpha=True,
    stops=True,
    min_token_length=3,
    min_tokens=2,
    vocab_file='1gram-corpus-vocab_list_match.txt',
    compress=True,
    overwrite=False,
    delete_input=True
)

### Sort and combine the multigram files
This modules creates a single, fully-sorted multigram file out of the filtered files. This is crucial for the next step (ngram consolidation; see below).   

Sorting a giant file is a resource-hungry process and I've tried to implement an efficient approach that leverages parallelism: We first sort the filtered files in parallel using Python's standard sorting algorithm [Timsort](https://en.wikipedia.org/wiki/Timsort); then, we incrementally [heapsort](https://en.wikipedia.org/wiki/Heapsort) the files in parallel until we get down to 2 files. Finally, we heapsort the final 2 files (necessarily using one processor) to arrive at a single combined and sorted unigram file.

Because this step can take a _very_ long time for larger multigrams (e.g., 5grams), we can run it in sessions using the `start_iteration` and `end_iteration` options. Iteration 1 comes after the initial file sort. If you only have time to complete, say, iterations 1–3, you can set `end_iteration=3`. During a later session, you can specify `start_iteration=4` to pick up where you left off.

`[Sort + Iteration 1 runtime: 1:24:28.705862]`
`[Iteration 2 runtime:        0:34:30.857246]`
`[Iteration 3 runtime:        0:41:38.534432]`
`[Iteration 4 runtime:        1:15:16.273906]`
`[Iteration 5 runtime:        2:28:44.848043]`
`[Iteration 6 runtime:        7:20:05.980459]`

In [ ]:
sort_ngrams(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    workers=12,
    sort_key='ngram',
    compress=True,
    overwrite=False,
    sort_order='ascending',
    start_iteration=6,
    end_iteration=6,
    delete_input=True
)

### Verify sort [OPTIONAL]
If we want, we can verify that the output file is correctly sorted. If the script outputs `True`, then the file is sorted. Bear in mind that you need to specify the file path manually here; be sure to use the right file extension based on whether `sort_ngrams` was run with `compress=True`.

In [ ]:
check_file_sorted(
    input_file=(
        '/vast/edk202/NLP_corpora/Google_Books/20200217/eng/'
        '5gram_files/6corpus/5gram-merged.jsonl.lz4'
    ),
    field="ngram",
    sort_order="ascending"
)

### Consolidate duplicate multigrams
This module consolidates the sorted multigram file. Lowercasing and lemmatizing produce duplicate unigrams. Now that the file is sorted, we can scan through it and consolidate consecutive idential duplicates. This involves summing their overall and yearly frequencies and document counts. It also leads to a much smaller file.

`[Runtime: 5:31:03.562892]`

In [ ]:
consolidate_duplicate_ngrams(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    lines_per_chunk=500000,
    compress=True,
    overwrite=False
)

### View line [OPTIONAL]
If we want, we can inspect a line in the file.

In [ ]:
print_jsonl_lines(
    file_path=(
        '/vast/edk202/NLP_corpora/Google_Books/20200217/eng/'
        '5gram_files/6corpus/yearly_files/2001.jsonl.lz4'
    ),
    start_line=100050,
    end_line=100060,
    parse_json=True
)

### Make yearly files
This module converts the overall corpus file into yearly corpora. For each year in which an ngram appeared, a `<year>.jsonl` file (or `<year>.jsonl.lz4` if `compress=True`) will be created. Each line in a yearly file contains an ngram, a `freq` value (the number of times it appeared that year), and a `doc` value (the number of unique documents it appeared in that year).

I found it difficult to prevent memory exhaustion when processing 5grams with 128GB of RAM. Users may have to reduce the number of processors and/or the `chunk_size` to stay within their limits. Also note that the final clean-up step, in which many temporary files get deleted, can take several minutes to complete. 

After creating yearly corpora, we can proceed to train `word2vec` models as shown in the `workflow_train_models.ipynb` notebook.

`[Runtime with compression: 2:15:47.004329]`

In [ ]:
make_yearly_files(
    ngram_size=5,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    overwrite=False,
    compress=True,
    workers=14,
    chunk_size=250000
)